In [1]:
import os
os.chdir("/tf/src")

In [2]:
!python preprocess.py -d squadca-qc

020-05-03 08:21:57.853846: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-05-03 08:21:57.887224: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2592000000 Hz
2020-05-03 08:21:57.889318: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5f3d890 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-05-03 08:21:57.889395: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-05-03 08:21:57.999046: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-03 08:21:57.999401: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5cf23b0 initialized for platform CUDA (this does not guarantee that XLA will be use

In [1]:
from text_gan.data.squad1_ca_qc import SQuAD_CA_QC
data = SQuAD_CA_QC()

In [3]:
len(list(data.train.as_numpy_iterator()))

81197

In [4]:
len(list(data.test.as_numpy_iterator()))

9560

In [1]:
from text_gan import cfg
from text_gan.features import GloVeReader, NERTagger, PosTagger
from text_gan.utils import MapReduce

import en_core_web_sm
from copynet_tf import Vocab
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import logging
import os
import gc

In [2]:
embedding_reader = GloVeReader()

In [3]:
pretrained_vectors = embedding_reader.read(cfg.EMBS_FILE)

Loading vectors: 2196017it [03:03, 11966.88it/s]


In [4]:
vocab = Vocab(
    embedding_reader.START,
    embedding_reader.END,
    embedding_reader.PAD,
    embedding_reader.UNK,
    cfg.CSEQ_LEN,
    cfg.QSEQ_LEN
)
ner = NERTagger(cfg.NER_TAGS_FILE, cfg.CSEQ_LEN)
pos = PosTagger(cfg.POS_TAGS_FILE, cfg.CSEQ_LEN)
nlp = en_core_web_sm.load()

In [5]:
train = tfds.load("squad", data_dir="/tf/data/tf_data", split='train').take(10)

In [10]:
def utf8_decoder(x):
        return x.decode('utf-8')

def substrSearch(ans, context):
    i = 0
    j = 0
    s = -1
    while i < len(context) and j < len(ans):
        if context[i].text == ans[j].text:
            s = i
            i += 1
            j += 1
        else:
            i += 1
            j = 0
            s = -1
    return s, j

In [18]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_context = train.map(
    lambda x: x['context'], num_parallel_calls=AUTOTUNE)
train_question = train.map(
    lambda x: x['question'], num_parallel_calls=AUTOTUNE)
train_ans = train.map(
    lambda x: x['answers']['text'][0], num_parallel_calls=AUTOTUNE)

Cause: could not parse the source code:

    lambda x: x['context'], num_parallel_calls=AUTOTUNE)

This error may be avoided by creating the lambda in a standalone statement.

Cause: could not parse the source code:

    lambda x: x['context'], num_parallel_calls=AUTOTUNE)

This error may be avoided by creating the lambda in a standalone statement.

Cause: could not parse the source code:

    lambda x: x['context'], num_parallel_calls=AUTOTUNE)

This error may be avoided by creating the lambda in a standalone statement.

Cause: could not parse the source code:

    lambda x: x['question'], num_parallel_calls=AUTOTUNE)

This error may be avoided by creating the lambda in a standalone statement.

Cause: could not parse the source code:

    lambda x: x['question'], num_parallel_calls=AUTOTUNE)

This error may be avoided by creating the lambda in a standalone statement.

Cause: could not parse the source code:

    lambda x: x['question'], num_parallel_calls=AUTOTUNE)

This error may be 

In [19]:
mr = MapReduce()
train_context = train_context.as_numpy_iterator()
train_context = mr.process(utf8_decoder, train_context)
train_question = train_question.as_numpy_iterator()
train_question = mr.process(utf8_decoder, train_question)
train_ans = train_ans.as_numpy_iterator()
train_ans = mr.process(utf8_decoder, train_ans)

In [20]:
train_context = nlp.pipe(
    train_context, batch_size=128, n_process=6)
train_question = nlp.pipe(
    train_question, batch_size=128, n_process=6)
train_ans = nlp.pipe(
    train_ans, batch_size=128, n_process=6)

In [21]:
training_context = []
training_question = []
training_ans = []
for context, ques, ans in zip(
        train_context, train_question, train_ans):
    ans_start, al = substrSearch(ans, context)
    ans_start += 1
    if len(ques) >= 20 or ans_start == -1 or ans_start + al >= 250:
        continue
    training_context.append(context)
    training_question.append(ques)
    ans = np.zeros(cfg.CSEQ_LEN, dtype=np.uint8)
    ans[ans_start:ans_start+al] = 1
    training_ans.append(ans)
len(training_context), len(training_question), len(training_ans)

(10, 10, 10)

In [22]:
vocab.fit(
    training_context,
    training_question,
    pretrained_vectors,
    0, 0
)
vocab.save(cfg.VOCAB_SAVE)
train_cidx = vocab.transform(training_context, "source")
train_ner = ner.transform(training_context)
train_pos = pos.transform(training_context)
train_qidx = vocab.transform(training_question, "target")


source max ('the', 88) min ('use.:74', 1)
target max ('?', 10) min ('for', 1)


In [23]:
train_cidx[0]

array([  2,  18, 168,   9,   4, 169, 170,  22,   4, 171,   7, 172,  16,
         4, 173,  23,  26, 174,  11,  76, 175,  22,   4, 176,  12, 177,
        77,  35,   6,  46, 178, 179,   4, 180,   5, 181,  78,  76, 182,
       183,   6,  46, 184, 185,   4,  79,  80,  22,  47, 186,  48, 187,
       188,  78,   6,  36,  81, 189, 190,   4,  79, 191, 192,  82,   9,
       193, 194,   5,  14,   9, 195, 196,   5,   4, 197,  83,  80, 198,
       199,  35,   5, 200,  14, 201,  35, 202,  37, 203,  11,  15,  38,
        77,   8, 204,  84,  12,  49, 205,  35,   6,   3,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [24]:
train_qidx[0]

array([ 2,  5,  6, 15, 16, 17, 18, 19, 20, 21,  8, 22, 23,  7, 24, 25, 26,
       27,  4,  3])

In [25]:
vocab._target['?']

4

In [26]:
training_ans[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [27]:
cseq = cfg.CSEQ_LEN
qseq = cfg.QSEQ_LEN

def gen():
    for cidx, ner, pos, qidx, ans in zip(
            train_cidx, train_ner, train_pos,
            train_qidx, training_ans):
        yield (cidx, ans, qidx, ner, pos)

train_dataset = tf.data.Dataset.from_generator(
    gen,
    (tf.int32, tf.uint8, tf.int32, tf.uint8, tf.uint8),
    (
        tf.TensorShape([cseq]), tf.TensorShape([cseq]),
        tf.TensorShape([qseq]), tf.TensorShape([cseq]),
        tf.TensorShape([cseq]))
)

In [28]:
i = 0
for cidx, ans, qidx, ner, pos in train_dataset:
    i += 1
print("Total", i)

Total 10
